In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings

import numpy as np
import gsw
import xarray as xr
from xgcm import Grid
from xhistogram.xarray import histogram
from dask.diagnostics import ProgressBar

import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
import regionmask

import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
import cmocean

import cartopy.crs as ccrs
import cartopy.feature as cfeature
data_crs = ccrs.PlateCarree()

In [4]:
import xwmb
import sectionate as sec
import regionate as reg

In [5]:
import dask

### Load grids and data

In [6]:
gridname = 'zstr'
dt = '6hourly' # choose from ['monthly', '6hourly', 'timestp']

# Load data on native grid
rootdir = '/archive/gam/MOM6-examples/ice_ocean_SIS2/Baltic_OM4_025/ocean_'+dt+'/'
prefix = '19000101.ocean_'+dt+'_'
time = "190*_*"

# Diagnostics were saved into different files
suffixs = ['thck','heat','salt','xtra']
Zprefixes = {'rho2':'rho2_', 'zstr':'z_', 'natv':'z'}
Zprefix = Zprefixes[gridname]
ds = xr.Dataset()
for suffix in suffixs:
    filename = prefix+gridname+'_'+suffix+'_'+time+'*.nc'
    dsnow = xr.open_mfdataset(rootdir+filename)
    ds = xr.merge([ds,dsnow])

# Load surface fluxes
filename = prefix+'natv_surf_'+time+'*.nc'
surf = xr.open_mfdataset(rootdir+filename)
ds = xr.merge([ds, surf])
    
# Load snapshot data (for mass tendency term)
suffix = 'snap'
filename = prefix+gridname+'_'+suffix+'_'+time+'*.nc'
snap = xr.open_mfdataset(rootdir+filename)

#  Load grid
oceangridname = '19000101.ocean_static.nc'
grid = xr.open_dataset(rootdir+oceangridname).squeeze()

# Cut off the first time-averaged entry in ds, because snap does not come with the initial condition
ds = ds.isel(time=slice(1,None))

# Some renaming to match hdrake conventions
ocean_grid = grid.copy().rename({'depth_ocean':'deptho'})
ds = ds.rename({'temp':'thetao'})
snap = snap.rename({'temp':'thetao'})

# z-coordinate dataset containing basic state variables
grid = Grid(ds, coords={'X': {'center': 'xh', 'right': 'xq'},
                        'Y': {'center': 'yh', 'right': 'yq'},
                        'Z': {'center': f'{Zprefix}l', 'outer': f'{Zprefix}i'} },
           periodic=None)

# z-coordinate dataset containing basic state variables
grid_snap = Grid(snap, coords={'X': {'center': 'xh',},
                        'Y': {'center': 'yh',},
                        'Z': {'center': f'{Zprefix}l', 'outer': f'{Zprefix}i'} },
           periodic=None)

In [7]:
ds = ds.sel(time=slice('1900-02-01 00', '1901-02-01 00'))
snap = snap.sel(time=slice('1900-02-01 00', '1901-02-01 00'))

## Just pick the first few timesteps for testing!!!

In [8]:
ds = ds.isel(time=[0])
ds.time.attrs = {}
snap = snap.isel(time=[0,1])
snap.time.attrs = {}

In [10]:
ds.to_netcdf("../data/baltic_example_ds.nc", mode="w", engine="netcdf4")

In [11]:
snap.to_netcdf("../data/baltic_example_snap.nc", mode="w", engine="netcdf4")